<a href="https://colab.research.google.com/github/mbubesa/ML-Algo-Exploration/blob/main/Pytorch_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A busy man's intro to Pytorch


In [1]:
import torch
import numpy as np

In [2]:
a = [1, 2, 3, 4, 5]
tensor_a = torch.tensor(a)
print(tensor_a)

tensor([1, 2, 3, 4, 5])


In [3]:
# From a NumPy array
b = np.array([[1, 2, 3], [4, 5, 6]])
tensor_b = torch.tensor(b)
print(tensor_b)


tensor([[1, 2, 3],
        [4, 5, 6]])


In [4]:
# Directly
# Float Tensor of Zeros
zeros_tensor = torch.zeros(2, 3)
print(zeros_tensor)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [5]:
# Integer tensor of Ones
ones_tensor = torch.ones(3, 2)
print(ones_tensor)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [6]:
# Random Float tensor
rand_tensor = torch.rand(4, 4)
print(rand_tensor)

tensor([[0.0113, 0.3585, 0.0946, 0.5681],
        [0.5980, 0.0564, 0.3976, 0.0049],
        [0.1919, 0.6588, 0.3327, 0.5968],
        [0.5428, 0.4096, 0.1525, 0.7741]])


In [7]:
# Tensor with Specific Values
values_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(values_tensor)

tensor([[1, 2, 3],
        [4, 5, 6]])


# Tensor Properties

In [8]:
print(tensor_a.shape)
print(tensor_b.size())

torch.Size([5])
torch.Size([2, 3])


# Data Types

In [9]:
# Specify Data Type
float_tensor = torch.tensor([1, 2, 3], dtype=torch.float32)
print(float_tensor)


tensor([1., 2., 3.])


In [10]:
# Change Data Type
int_tensor = float_tensor.int()
print(int_tensor.dtype)

torch.int32


# Let's do some modeling and train a simple model

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 18387806.42it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 331497.83it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6154243.34it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4483446.22it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [13]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [14]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
# Optimizing model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [18]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
------------------------------------------
loss: 2.304898  [   64/60000]
loss: 2.294736  [ 6464/60000]
loss: 2.276637  [12864/60000]
loss: 2.274745  [19264/60000]
loss: 2.267255  [25664/60000]
loss: 2.220949  [32064/60000]
loss: 2.244377  [38464/60000]
loss: 2.203438  [44864/60000]
loss: 2.196180  [51264/60000]
loss: 2.170063  [57664/60000]
Epoch 2
------------------------------------------
loss: 2.175681  [   64/60000]
loss: 2.166236  [ 6464/60000]
loss: 2.109340  [12864/60000]
loss: 2.123245  [19264/60000]
loss: 2.092553  [25664/60000]
loss: 2.013007  [32064/60000]
loss: 2.048468  [38464/60000]
loss: 1.967497  [44864/60000]
loss: 1.967774  [51264/60000]
loss: 1.896317  [57664/60000]
Epoch 3
------------------------------------------
loss: 1.932450  [   64/60000]
loss: 1.902502  [ 6464/60000]
loss: 1.784444  [12864/60000]
loss: 1.821101  [19264/60000]
loss: 1.741509  [25664/60000]
loss: 1.663945  [32064/60000]
loss: 1.689509  [38464/60000]
loss: 1.587737  [44864/60000]
loss: 1